# Product allocation

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=examples/product-allocation.ipynb">directly from your browser</a>.
</div>

This notebook implements an optimization model for allocating retail products to stores given demand, supply, and diversity constraints.

In [1]:
%pip install opvious

## Model

We first formulate the model using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html). You can also find an equivalent LaTeX formulation [here](https://github.com/opvious/examples/blob/main/sources/product-allocation.md).

In [2]:
import opvious.modeling as om

class ProductAllocation(om.Model):
    products = om.Dimension()
    sizes = om.Dimension()
    locations = om.Dimension()
    tiers = om.Dimension()

    supply = om.Parameter.natural(products, sizes, name=r"a^\mathrm{max}")
    min_allocation = om.Parameter.natural(products)
    max_total_allocation = om.Parameter.natural()
    breadth = om.Parameter.natural(products)
    demand = om.Parameter.natural(locations, products, sizes, tiers)
    value = om.Parameter.continuous(tiers)

    allocation = om.Variable.natural(*demand.quantifiables(), upper_bound=max_total_allocation())
    product_allocated = om.fragments.ActivationVariable(allocation, projection=0b11)
    size_allocated = om.fragments.ActivationVariable(allocation, projection=0b111, upper_bound=False, lower_bound=1)
        
    @om.constraint
    def allocation_fits_within_demand(self):
        for l, p, s, t in self.locations * self.products * self.sizes * self.tiers:
            yield self.allocation(l, p, s, t) <= self.demand(l, p, s, t)
            
    @om.constraint
    def allocation_fits_within_supply(self):
        for p, s in self.products * self.sizes:
            yield om.total(self.allocation(l, p, s, t) for l, t in self.locations * self.tiers) <= self.supply(p, s)
            
    @om.constraint
    def total_allocation_fits_within_max(self):
        total = om.total(
            self.allocation(l, p, s, t)
            for l, p, s, t in self.locations * self.products * self.sizes * self.tiers
        )
        yield total <= self.max_total_allocation()
        
    @om.constraint
    def allocation_meets_product_min(self):
        for p, l in self.products * self.locations:
            alloc = om.total(self.allocation(l, p, s, t) for s, t in self.sizes * self.tiers)
            yield alloc >= self.min_allocation(p) * self.product_allocated(l, p)
        
    @om.constraint
    def allocation_meets_product_breadth(self):
        for p, l in self.products * self.locations:
            breadth = om.total(self.size_allocated(l, p, s) for s in self.sizes)
            yield breadth >= self.breadth(p) * self.product_allocated(l, p)

    @om.objective
    def maximize_value(self):
        return om.total(
            self.value(t) * self.allocation(l, p, s, t)
            for l, p, s, t in self.locations * self.products * self.sizes * self.tiers
        )

    
model = ProductAllocation()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">ProductAllocation</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{products}&: P \\
  \S^d_\mathrm{sizes}&: S \\
  \S^d_\mathrm{locations}&: L \\
  \S^d_\mathrm{tiers}&: T \\
  \S^p_\mathrm{supply}&: a^\mathrm{max} \in \mathbb{N}^{P \times S} \\
  \S^p_\mathrm{minAllocation}&: a^\mathrm{min} \in \mathbb{N}^{P} \\
  \S^p_\mathrm{maxTotalAllocation}&: a^\mathrm{maxTotal} \in \mathbb{N} \\
  \S^p_\mathrm{breadth}&: b \in \mathbb{N}^{P} \\
  \S^p_\mathrm{demand}&: d \in \mathbb{N}^{L \times P \times S \times T} \\
  \S^p_\mathrm{value}&: v \in \mathbb{R}^{T} \\
  \S^v_\mathrm{allocation}&: \alpha \in \{0 \ldots a^\mathrm{maxTotal}\}^{L \times P \times S \times T} \\
  \S^v_\mathrm{productAllocated}&: \alpha^\mathrm{product} \in \{0, 1\}^{L \times P} \\
  \S^c_\mathrm{productAllocatedActivates}&: \forall l \in L, p \in P, s \in S, t \in T, a^\mathrm{maxTotal} \alpha^\mathrm{product}_{l,p} \geq \alpha_{l,p,s,t} \\
  \S^v_\mathrm{sizeAllocated}&: \alpha^\mathrm{size} \in \{0, 1\}^{L \times P \times S} \\
  \S^c_\mathrm{sizeAllocatedDeactivates}&: \forall l \in L, p \in P, s \in S, \alpha^\mathrm{size}_{l,p,s} \leq \sum_{t \in T} \alpha_{l,p,s,t} \\
  \S^c_\mathrm{allocationFitsWithinDemand}&: \forall l \in L, p \in P, s \in S, t \in T, \alpha_{l,p,s,t} \leq d_{l,p,s,t} \\
  \S^c_\mathrm{allocationFitsWithinSupply}&: \forall p \in P, s \in S, \sum_{l \in L, t \in T} \alpha_{l,p,s,t} \leq a^\mathrm{max}_{p,s} \\
  \S^c_\mathrm{totalAllocationFitsWithinMax}&: \sum_{l \in L, p \in P, s \in S, t \in T} \alpha_{l,p,s,t} \leq a^\mathrm{maxTotal} \\
  \S^c_\mathrm{allocationMeetsProductMin}&: \forall p \in P, l \in L, \sum_{s \in S, t \in T} \alpha_{l,p,s,t} \geq a^\mathrm{min}_{p} \alpha^\mathrm{product}_{l,p} \\
  \S^c_\mathrm{allocationMeetsProductBreadth}&: \forall p \in P, l \in L, \sum_{s \in S} \alpha^\mathrm{size}_{l,p,s} \geq b_{p} \alpha^\mathrm{product}_{l,p} \\
  \S^o_\mathrm{maximizeValue}&: \max \sum_{l \in L, p \in P, s \in S, t \in T} v_{t} \alpha_{l,p,s,t} \\
\end{align*}
$$
</div>
</details>
</div>

## Testing

Let's solve the model above on a small dataset.

In [3]:
import io
import opvious
import pandas as pd

client = opvious.Client.default("https://try.opvious.io")

In [4]:
demand_df = pd.read_csv(io.StringIO("""
location,tier,product,size,demand
Boston,T1,hoodie,M,50
Boston,T1,shirt,L,30
Boston,T2,shirt,L,25
Boston,T1,shirt,XL,20
Seattle,T1,hoodie,M,100
Seattle,T1,hoodie,L,75
Seattle,T2,hoodie,L,50
Seattle,T1,hoodie,XL,50
Seattle,T1,shirt,L,10
""")).set_index(["location", "product", "size", "tier"])
demand_df

demand
location product size tier        
Boston   hoodie  M    T1        50
         shirt   L    T1        30
                      T2        25
                 XL   T1        20
Seattle  hoodie  M    T1       100
                 L    T1        75
                      T2        50
                 XL   T1        50
         shirt   L    T1        10

In [5]:
supply_df = pd.read_csv(io.StringIO("""
product,size,supply
hoodie,M,100
hoodie,L,50
shirt,L,50
shirt,XL,10
""")).set_index(["product", "size"])
supply_df

supply
product size        
hoodie  M        100
        L         50
shirt   L         50
        XL        10

In [6]:
product_df = pd.read_csv(io.StringIO("""
product,min_allocation,diversity
hoodie,100,2
shirt,10,2
""")).set_index(["product"])
product_df

,min_allocation,diversity
product,,
hoodie,100,2
shirt,10,2


In [7]:
solution = await client.solve(
    opvious.Problem(
        specification=model.specification(),
        parameters={
            "demand": demand_df["demand"],
            "value": {"T1": 1, "T2": 0.8},
            "minAllocation": product_df["min_allocation"],
            "breadth": product_df["diversity"],
            "supply": supply_df["supply"],
            "maxTotalAllocation": 500
        },
    ),
)

In [8]:
solution.outputs.variable("allocation")

value
locations products sizes tiers       
Boston    shirt    L     T1        30
                         T2        20
                   XL    T1        10
Seattle   hoodie   L     T1        50
                   M     T1       100